In [15]:
##Packages:
import sys
import string
import os
import glob
import math
import numpy as np 
import hdf5storage as hdf
import scipy as sp
import pandas as pd
import nilearn
import nibabel as nb
from nilearn import plotting, image, masking, regions, datasets
import io
import time
import subprocess
import sys
import natsort
import pyxnat
from pyxnat import Interface

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.colors as mcolors
import matplotlib.image as mpimg
from termcolor import colored, cprint
import multiprocessing

from time import time
from random import random

## nimlab package is hosted internally on the M Falcon
#from nimlab.functions import *
from nimlab.software import *
#from nimlab.xnat_functions import *

## Environment:

MNI_brain_mask=datasets.load_mni152_brain_mask()
MNI_nonbrain_mask=image.math_img("-img+1", img=MNI_brain_mask)

In [16]:

#input_folder='/data2/jupyter/notebooks/lsoussan/forge2/Caen/Caen_group1/test_lessions/'
input_folder='/home/clancy/data/LesionBrainMapping/ROIs/'
PROJECT_NAME_FOR_XNAT=''

preprocess_folder = '/home/clancy/data/LesionBrainMapping/test_kang/'

lesion_files=glob.glob(input_folder+'/*nii.gz')
lesion_files

# Create Preprocessing folder
if preprocess_folder:    
    if not os.path.exists(preprocess_folder+'Originals/'):
        os.makedirs(preprocess_folder+'Originals/')
        #final ROIs (not lesions)
    if not os.path.exists(preprocess_folder+'Lesions/'):
        os.makedirs(preprocess_folder+'Lesions/')
        #Analyses: empty subfolder for subsequent analyses
    if not os.path.exists(preprocess_folder+'Functional_Connectivity/'):
        os.makedirs(preprocess_folder+'Functional_Connectivity/')
    if not os.path.exists(preprocess_folder+'Connectome/'):
        os.makedirs(preprocess_folder+'Connectome/')
else:
    preprocess_folder=os.getcwd()+'/Preprocessing'
    if not os.path.exists('./Preprocessing'):
        os.makedirs('./Preprocessing')
        os.makedirs('./Preprocessing/Originals')
        os.makedirs('./Preprocessing/Lesions')
        os.makedirs('./Preprocessing/Functional_Connectivity')
        os.makedirs('./Preprocessing/Connectome')

# Standardizing the lesions: 

In [18]:
new_lesion_files = []
subjects_list = []
filenames_list=[]
#lcs=longest_common_suffix(lesion_files)
import re
import shutil
#Warning: lesion filenames should not have "." in them aside from designating extension

for i in lesion_files:
    shutil.copyfile(i, preprocess_folder + '/Originals/' + i.split('/')[-1])
    filename = os.path.basename(i)
    filenames_list.append(filename)
    filepath = os.path.dirname(i)
    #subject_name = filename.split(lcs)[0]
    subject_name = re.findall(r'(sub\d+)_', filename)[0]
    subjects_list.append(subject_name)
    preprocess_filename = subject_name + '_lesion.nii.gz'
    new_filepath =  preprocess_folder + '/Lesions/' + preprocess_filename
    new_lesion_files.append(new_filepath)
    # print subject_name
    print('Reslicing to FSL MNI 2mm Template and binarizing ' + filename)
    
    !flirt -in $i -ref /usr/share/fsl/5.0/data/standard/MNI152_T1_2mm.nii.gz -datatype float -interp spline -applyxfm -usesqform -out $new_filepath
    
    !fslmaths $new_filepath -thr 0.25 -bin $new_filepath -odt short

Reslicing to FSL MNI 2mm Template and binarizing sub001_ROI-1.nii.gz
Image Exception : #63 :: No image files match: /usr/share/fsl/5.0/data/standard/MNI152_T1_2mm
Image Exception : #22 :: Failed to read volume /usr/share/fsl/5.0/data/standard/MNI152_T1_2mm.nii.gz
Error : No image files match: /usr/share/fsl/5.0/data/standard/MNI152_T1_2mm
Failed to read volume /usr/share/fsl/5.0/data/standard/MNI152_T1_2mm.nii.gz
Error : No image files match: /usr/share/fsl/5.0/data/standard/MNI152_T1_2mm
Image Exception : #63 :: No image files match: /home/clancy/data/LesionBrainMapping/test_kang//Lesions/sub001_lesion
terminate called after throwing an instance of 'std::runtime_error'
  what():  No image files match: /home/clancy/data/LesionBrainMapping/test_kang//Lesions/sub001_lesion
Reslicing to FSL MNI 2mm Template and binarizing sub002_ROI-2.nii.gz
Image Exception : #63 :: No image files match: /usr/share/fsl/5.0/data/standard/MNI152_T1_2mm
Image Exception : #22 :: Failed to read volume /usr/sha